In [ ]:
if True: # enable folding code
    if False:
        from julia.api import Julia
        jl = Julia(compiled_modules=False)

    import julia; julia.install(quiet=True)
    from julia import Main

    import numpy as np
    import sympy as sp
    from IPython.display import display, Math

    np.set_printoptions(precision=3, suppress=True)

    try:
        from scipy.linalg import cholesky, eigh, qz, ordqz
        HAVE_SCIPY = True
    except Exception as e:
        HAVE_SCIPY = False
        print("SciPy not available; QZ and ordqz demos will be skipped:", e)

    def residual(A,B,lam,x):
        num = np.linalg.norm(A@x - lam*(B@x))
        den = (np.linalg.norm(A)*np.linalg.norm(x) + 1e-15)
        return num/den

In [ ]:
%load_ext julia.magic

In [194]:
%%julia
using Pkg
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, BlockArrays, RowEchelon, LaTeXStrings, Latexify, SymPy, Random

using PyCall
itikz = pyimport("itikz")
nM    = pyimport("itikz.nicematrix");
# -------------------------------------------------------------------------------------------------------
function naive_householder( A; tol = :none )
    # naive implementation from HouseholderRlections.ipynb
    e = eltype(A)
    if tol == :none tol = zero(e) end

    function sgn(x)
        x != zero(e) ? sign(x) : one(e)
    end

    m,n = size(A)
    AI  = [copy(A) one(e)I(m)]

    matrices = [ [:none, AI ] ]

    i = 1; j = 1
    while i < m && j <= n
        v         = AI[:,j]                                  # original vector
        v[1:i-1] .= zero(e)                                  # keep only the part of the vector we are working with
        len_v     = sqrt(v'v)
        if (len_v - abs(v[i])) > tol                         # non-zero values below current pivot position
            ṽ     = zeros(e,m); ṽ[i] = - sgn(v[i]) * len_v   # reflected vector (if v[j] = 0, we set the sign to 1)

            w     = v + ṽ                                    # bisector
            Q     = I(m) - 2*w*w' / (w'w)                    # reflection matrix  (note this assumes v ≠ 0)
            AI    = Q*AI                                     # update the matrix

            push!(matrices, [Q, copy(AI)] )
            i += 1
        end
        j += 1
    end

    return matrices
end

function lower_hessenberg_formatter(x, i, j, formatted_x; colmax=1)
    # color entries below the first subdiagonal in cols 1:colmax
    if (i > j + 1) && (j <= colmax)
        return "\\textcolor{red}{$formatted_x}"
    else
        return formatted_x
    end
end

function tril_formatter(x, i, j, formatted_x;
                        k::Int = 0,               # offset from diagonal
                        color::String = "red",
                        c1::Int = 1,              # first column
                        c2::Int = typemax(Int))   # last column
    if (i >= j - k) && (c1 <= j <= c2)
        return "\\textcolor{$color}{$formatted_x}"
    else
        return formatted_x
    end
end


function block_formatter(x, i, j, formatted_x; r1=1, r2=1, c1=1, c2=1)
    # color entries in selected block in red
    if (r1 <= i <= r2) && (c1 <= j <= c2)
        return "\\textcolor{red}{$formatted_x}"
    else
        return formatted_x
    end
end
;

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Generalized Eigenvalue Computation</strong>
</div></div>

# 1. Introduction

This notebook addresses numerical methods for solving the generalized eigenproblem (GEP):  
$\qquad \displaystyle{ A \mathbf{v} = \lambda B \mathbf{v} }$

The focus is on algorithmic strategies for computing eigenvalues and eigenvectors,  
particularly in structured cases where $A$ and $B$ are symmetric, and $B$ is positive definite.  
Structured solvers are contrasted with general-purpose routines suitable for arbitrary pencils.

Topics include:

- Solvers for symmetric-definite pencils
- Treatment of general matrix pencils
- Sensitivity to conditioning and numerical roundoff
- Interpretation of computed eigenpairs

The theoretical foundation, including variational formulations and physical interpretations,  
is presented in the preceding notebook: [**GEP_intro.ipynb**](GEP_intro.ipynb).

# 2. Symmetric-Definite Case

Consider the generalized eigenproblem  
$\qquad \displaystyle{ A x = \lambda B x}$  
where both matrices $A$ and $B$ are real symmetric, and $B$ is **positive definite**,  
i.e., $A=A^T$, $B=B^T \succ 0$

**Remark:** when $B$ is invertible, we may convert the GEP to a standard eigenproblem:  
$\qquad A x = \lambda B x \;\Leftrightarrow\; B^{-1} A x = \lambda x$.  
$\qquad$ Numerically however, forming $B^{-1} A$ explicitly is **dangerous** when $B$ is ill-conditioned;
it also destroys structure.  

$\qquad$ We will therefore formulate an algorithm that avoids inverting $B$ using the Cholesky Factorization.

____

Since $B$ is symmetric positive definite, it admits a **Cholesky decomposition**:  
$\qquad \displaystyle{ B = L^T L }$  
where $L$ is upper triangular with positive diagonal entries.

Substituting into the generalized eigenproblem:  
$\qquad \displaystyle{ A x = \lambda\ L^T L x }$

Let $\;\; \displaystyle{ x = L^{-1} z }$, then  
$\qquad \displaystyle{
\begin{aligned}
A x = \lambda B x 
\quad &\Leftrightarrow\quad A L^{-1} z = \lambda L^T L L^{-1} z \\
     &\Leftrightarrow\quad A L^{-1} z = \lambda L^T z \\
     &\Leftrightarrow\quad (L^T)^{-1} A L^{-1} z = \lambda z
\end{aligned}
}$

Define $\;\; \displaystyle{ \tilde{A} = (L^T)^{-1} A L^{-1} }$.  
Then the problem reduces to the standard eigenproblem:  
$\qquad \displaystyle{ \tilde{A} z = \lambda z }$

Once the eigenpairs $(\lambda, z)$ are computed, the original eigenvectors are obtained by  
$\qquad \displaystyle{ x = L^{-1} z }$

**Remark:** **explicit computation of** $\displaystyle{ L^{-1} }$ **or** $\displaystyle{ (L^T)^{-1} }$ **should be avoided in practice**.
- To evaluate $\displaystyle{x = L^{-1} z }$, compute $z$ via forward substitution (i.e., solve $L x = z$)
- To apply the transformation $\displaystyle{ \tilde{A} = (L^T)^{-1} A L^{-1} }$,
  solve two triangular systems rather than forming the inverse

#### **Algorithm Summary**

1) Compute $L$ (Cholesky factorization of $B$).  
2) Form $\tilde A = L^{-1} A L^{-T}$ by triangular solves.  
3) Compute eigenpairs $(\lambda, z)$ of the symmetric matrix $\tilde A$.  
4) Recover $x = L^{-T} z$; optionally $B$-normalize: $x \leftarrow x/\sqrt{x^T B x}$.

In [123]:
def gep_symmetric_definite(A,B):
    """Solve A x = λ B x for symmetric A and SPD B via Cholesky whitening.
    Returns (lam, X) with columns of X being B-normalized eigenvectors.
    """
    if not HAVE_SCIPY:
        raise RuntimeError("SciPy required here for robust Cholesky/eigh.")
    L = cholesky(B, lower=True)
    # Solve for At = L^{-1} A L^{-T}
    At = np.linalg.solve(L, A)
    At = At @ np.linalg.solve(L.T, np.eye(A.shape[0]))
    lam, Z = eigh(At)
    # Map back and B-normalize
    X = np.linalg.solve(L.T, Z)
    for i in range(X.shape[1]):
        xi = X[:,i]
        nB = np.sqrt(xi.T @ B @ xi)
        if nB > 0:
            X[:,i] = xi / nB
    return lam, X

#### **Example**

In [124]:
def sym_pos_def_demo():
    #np.random.seed(0)
    n = 5
    R = np.random.randn(n,n)
    B = R@R.T + n*np.eye(n)       # SPD
    S = np.random.randn(n,n)
    A = 0.5*(S+S.T)
    lam, X = gep_symmetric_definite(A,B)

    A_rep = sp.latex(sp.Matrix(np.round(A,2)), mat_str='pmatrix', mat_delim=None)
    B_rep = sp.latex(sp.Matrix(np.round(B,2)), mat_str='pmatrix', mat_delim=None)
    display(Math("A = " + A_rep + r",\quad B = " + B_rep))

    print("\nEigenvalues (symmetric-definite route):", lam)
    print("\n    ||X^T B X - I||         = ", np.linalg.norm(X.T @ B @ X - np.eye(n)))
    print("    ||A X - B X diag(lam)|| = ", np.linalg.norm(A@X - B@X@np.diag(lam)))
sym_pos_def_demo()

<IPython.core.display.Math object>


Eigenvalues (symmetric-definite route): [-0.266 -0.066 -0.035  0.219  0.367]

    ||X^T B X - I||         =  2.0680439868174287e-15
    ||A X - B X diag(lam)|| =  3.4216702701594093e-15


# 3. General Case

## 3.1 Matrices in Generalized Schur Form

<div style="background-color:#F2F5A9;color:black;">

**Definition:**  A pair $(S,T)$ is in **(real, quasi-) generalized Schur form**<br>
$\qquad$ if both are block upper triangular with the **same** block pattern<br>
$\qquad$ and each diagonal block is either $1\times1$ or $2\times2$.

The generalized eigenproblem for $(S,T)$ is $\;\; S x=\lambda\,T x$.
</div>

**This immediately yields eigenvalues:**  
Since $(S,T)$ are triangular by blocks, $\det(S-\lambda T)$ factors over the diagonal blocks:
- For a $1\times1$ block $([s_{ii}],[t_{ii}])$, eigenvalue $\,\lambda_i=s_{ii}/t_{ii}$ (finite if $t_{ii}\neq 0$, infinite if $t_{ii}=0$).
- For a $2\times2$ block $(S_{kk},T_{kk})$, the two eigenvalues are the roots of
<br>$\qquad \det(S_{kk}-\lambda T_{kk})=0$.

**This yields eigenvectors.**  
Fix an eigenvalue $\lambda$. Solve the homogeneous system
<br>$\qquad (S-\lambda T)x=0$
by **block back substitution**:
- If the trailing active block is $1\times1$, pick and normalize $x_i$, then solve upward.
- If it is $2\times2$, solve $(S_{kk}-\lambda T_{kk})\,x_{[k:k+1]}=0$, then continue upward.
Left eigenvectors satisfy $w^\top S=\lambda\, w^\top T$ and are obtained the same way on the transposed pair.

#### Example

Consider
$\;\;
S=\left(\begin{array}{rrrr}
\color{red}4&\color{red}7&5&6\\
\color{red}{-4}&\color{red}{-2}&3&8\\
0&0&\color{red}6&2\\
0&0&0&\color{red}{-3}
\end{array}\right),\quad
T=\begin{pmatrix}
\color{red}2&\color{red}1&4&3\\
\color{red}0&\color{red}2&7&5\\
0&0&\color{red}2&1\\
0&0&0&\color{red}1
\end{pmatrix}\;\;
$ with the 3 blocks shown in red.

**Eigenvalues.**
- Leading $2\times2$ block $\;\;S_{1} = \left(\begin{array}{rr} 4&7 \\ {-4}&{-2}\\ \end{array}\right), \;\;
T_{1} = \begin{pmatrix} 2 & 1 \\ 0 & 2\end{pmatrix}$

$\qquad \det\!\big(S_{1}-\lambda T_{1}\big)=4(\lambda^2-2\lambda+5)=0
\;\Rightarrow\; \lambda=1\pm 2i.$
- $1\times1$ blocks $\;\;S_2=\begin{pmatrix}6\end{pmatrix},\;\; T_2 = \begin{pmatrix}2\end{pmatrix}\;\;$ and
$S_3=\begin{pmatrix}-3\end{pmatrix},\;\; T_3 = \begin{pmatrix}1\end{pmatrix}\;\;$

$\qquad \lambda_3=6/2=3,\quad \lambda_4=-3/1=-3.$

**Right eigenvectors (Schur basis).** For each listed $\lambda$, solve
<br>$\qquad (S-\lambda T) x = 0$
by back substitution as above:

$\qquad
x_{1-2i} =
\left(\begin{array}{c}
-1 + i\\
1\\
0\\
0
\end{array}\right),
\;\;
x_{1+2i} =
\left(\begin{array}{c}
-1 - i\\
1\\
0\\
0
\end{array}\right),
\;\;
x_{3} =
\left(\begin{array}{r}
-16\\
-1\\
4\\
0
\end{array}\right),
\;\;
x_{-3} =
\left(\begin{array}{r}
59\\
-97\\
-20\\
48
\end{array}\right).
$

## 3.2 The A = RQ and the A = QR Factorizations

A QR Factorization may be used to obtain a RQ factorization of a matrix

Let $J=\begin{pmatrix} 0&\cdots&0&\color{red}1\\ 0&\cdots&\color{red}1&0\\ \vdots & \ddots&\vdots&\vdots\\ \color{red}1&0&\cdots&0 \end{pmatrix}\;\;$
be the reversal (permutation) matrix, i.e. $J$ flips the order of rows (or columns).

Because $J$ is orthogonal we have $J^{T}=J^{-1}=J$.

We may is this matrix to obtain a factorization $A = \tilde{A}\ \tilde{Q}$,<br>
where $\tilde{A}$ is upper triangular and $\tilde{Q}$ is orthogonal (or unitary)<br>
as follows:

Starting from the QR factorisation of the reflected transpose $J^{T}A^{T}J = Q R$,
we obtain <br>
$\qquad \begin{align}
J^T A^T J = Q R\quad  & \Leftrightarrow \quad A^T  = J Q J^T J R J^T  & \\
                & \Leftrightarrow \quad A^T  = \tilde{Q} \tilde{R} \qquad\qquad \text{ where }                               \tilde{Q} = J Q J^T, \tilde{R} = J R J^T \\
                & \Leftrightarrow \quad A = \tilde{R}^T \tilde{Q}^T &
\end{align}$

### 3.2.1  Example: $A = R Q$

In [ ]:
%%julia
function naive_rq( M; name=nothing)
    P = 1I(size(M, 1))[end:-1:1, :]
    M̃ = P * transpose(M) * P
    qr_fact = qr(M̃)

    Qt = Matrix(qr_fact.Q)             # orthogonal factor (explicit dense matrix)
    Rt = Matrix(qr_fact.R)             # upper‑triangular factor (explicit dense matrix)

    B = P * transpose(Rt) * P
    C = P * transpose(Qt) * P

    if ! isnothing(name)
        display(py_show( "Decompose ", name, " = ", Int.(M), L",\quad J = ", P, L",\quad \text{ so }\; J ", name, " J = ", Int.(M̃)))
    end

    B, C
end

function test_naive_rq()
    A = [4.0 1.0 2.0; 2.0 3.0 1.0; 1.0 0.0 2.0]

    R,Q = naive_rq(A; name="A")
    println()
    display(py_show( L"A = RQ, \text{ where } R \approx ", R, L"\quad Q \approx ", Q, number_formatter=x->round_value(x,2)))
    println("Check:")
    @show (B ≈ R*Q); flush(stdout)
    @show transpose(Q)*Q ≈ I;
end
test_naive_rq();

### 3.2.2 $A = RQ$ one Column at a Time

We will need to zero out entries in a matrix one column at a time.<br>
To do so, we have to zero out entries in a row to the left of a given position in transformed space.

Below a naive reference implementation using Householder transformations to zero out entries<br>
$\qquad$ **one column at a time**

In [126]:
%%julia
@doc """
householder_vector(x::AbstractVector; target::Symbol = :first) -> Vector

Computes a Householder vector `v` such that the reflection 
    `Q = I - 2vvᵀ` maps `x` to a vector aligned with the `target` coordinate.

Arguments:
- `x`: A vector to be reflected.
- `target`: `:first` (default) aligns with the first coordinate; `:last` aligns with the last.

Returns:
- A unit Householder vector `v` such that `Qx` has all zeros except possibly one entry.
"""
function householder_vector(x::AbstractVector; target::Symbol = :first)
    x = vec(x)
    if all(iszero, x)
        return zeros(eltype(x), length(x))
    end

    σ        = norm(x)
    e        = zeros(eltype(x), length(x))
    index    = target == :first ? 1 : length(x)
    e[index] = sign(x[index]) == 0 ? σ : sign(x[index]) * σ

    v     = x - e
    vnorm = norm(v)

    if vnorm ≈ 0
        return zeros(eltype(x), length(x))
    end

    result = v / vnorm
    return result
end
;

In [127]:
%%julia
"""
    QR_step!(A::Matrix{Float64}, i::Int, j::Int) -> Matrix

Applies a Householder reflection **from the left** to zero out entries
**below** the pivot `A[i,j]` in column `j`. That is, it modifies `A`
in-place as `A ← Q * A`, where `Q` is orthogonal.

Returns the orthogonal matrix `Q` such that the updated `A = Q * A`
has zeros in entries A[i+1:end, j].

Arguments:
- `A`: Matrix to modify in-place.
- `i`: Row index of the pivot.
- `j`: Column index of the pivot.

Returns:
- The orthogonal matrix `Q` such that `A ← Q * A`.
"""
function QR_step!(A::AbstractMatrix{Float64}, i::Int, j::Int)
    m = size(A, 1)

    # Extract column segment below and including the pivot
    x     = copy(@view A[i:end, j])
    v_sub = householder_vector(x; target = :first)  # Align with first coordinate

    v = zeros(eltype(A), m)
    v[i:end] .= v_sub

    Q = I - 2 * (v * v')  # Householder matrix
    A .= Q * A            # Apply from the left

    return Q
end;

In [ ]:
%%julia

function RQ_step!(
    A::AbstractMatrix{Float64},
    i::Int,              # pivot row index
    j::Int;              # pivot column index
    len::Int = j         # number of active entries (default: full 1:j)
)
    n = size(A, 2)
    if len <= 0
        return Matrix{Float64}(I, n, n)
    end

    start_col = max(1, j - len + 1)
    x = @view A[i, start_col:j]          # read directly from A

    # nothing to eliminate?
    #if all(iszero, x)
    #    return Matrix{Float64}(I, n, n)
    #end

    v_sub = householder_vector(x; target = :last)

    v = zeros(eltype(A), n)
    v[start_col:j] .= v_sub

    Qr = I - 2 * (v * v')
    A .= A * Qr
    return Qr
end

### Hessenberg–Triangular Reduction (Householder-based)

The **Hessenberg–triangular (HT) reduction** transforms a matrix pair $(A,B)$ into an equivalent pair $(H,T)$, where $H$ is upper Hessenberg and $T$ is upper triangular:

$$
Q^{T} A Z = H, \qquad Q^{T} B Z = T.
$$

---

### Algorithm (concise description)

1. **Triangularize $B$:**

   Apply a sequence of *left* Householder reflections $Q_k$ (“QR steps”) to $B$ until it becomes upper triangular:

   $$
   B \leftarrow Q_k B, \qquad  A \leftarrow Q_k A.
   $$

2. **Form Hessenberg–triangular pair:**

   For each $k = 1,\dots,n-2$:
   - Use a *left* Householder ($Q_k$) to zero entries below $A_{k+1,k}$;  
     apply it to both $A$ and $B$.
   - The left update introduces a *bulge* just below the diagonal of $B$.  
     Eliminate this bulge by applying *right* Householder reflections ($Z_i$, “RQ steps”) **from bottom to top**, with the reflection length decreasing upward:
     $$
     i = n, n-1, \dots, k+2, \qquad
     Z_i \text{ acts on row } i \text{ with length } (i-k).
     $$
     Each $Z_i$ removes one subdiagonal element of $B$ while updating $A$:
     $$
     B \leftarrow B Z_i, \qquad  A \leftarrow A Z_i.
     $$

3. **Finish:**

   After $n-2$ iterations,  
   $A$ is upper Hessenberg, $B$ is upper triangular, and  
   the accumulated matrices $Q=\prod Q_k$, $Z=\prod Z_i$ satisfy  

   $$
   H = Q^{T} A Z, \qquad T = Q^{T} B Z.
   $$

---

This two-sided process alternates left “QR” reductions and right “RQ” bulge chases, progressively constructing the upper‑Hessenberg/upper‑triangular pair that serves as the starting point of the **QZ (Generalized Schur) algorithm**.


In [207]:
%%julia
"""
Educational Hessenberg–Triangular reduction for a matrix pair (A,B).

Uses:
  • QR_step!(A, i, j) – left Householder; zeros entries below A[i,j].
  • RQ_step!(A, i, j) – right Householder; zeros entries left of A[i,j].

Both functions modify the given matrix in place and return the full N×N orthogonal Q or Z.

Performs:
    Qᵗ * A * Z = H
    Qᵗ * B * Z = T
where H is upper Hessenberg and T is upper triangular.
"""
function hessenberg_triangular_reduction!(A::Matrix{Float64}, B::Matrix{Float64})
    n = size(A, 1)
    Q_total = Matrix{Float64}(I, n, n)
    Z_total = Matrix{Float64}(I, n, n)

    # ---------------------------------------------------------
    # Step 1 : Triangularize B (QR-type left transformations)
    # ---------------------------------------------------------
    println("=========== Step 1 : Triangularize B from the left (QR_steps)")
    for j in 1:n-1
        Q = QR_step!(B, j, j)      # internally: B ← Q B
        A .= Q * A                 # apply Q to A once
        Q_total .= Q * Q_total
    end
    py_show( (a=A,),
             (b=B, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx; k=-1, c2=n) ),
             number_formatter=x->round(x,digits=2)
    )

    # ---------------------------------------------------------
    # Step 2 : Hessenberg–Triangular Reduction
    # ---------------------------------------------------------
    println("\n=========== Step 2 : Hessenberg–Triangular Reduction ===")
    for k in 1:n-2
        # ---- Left reflector Qₖ: zero below A[k+1,k]
        println("Make column $k of A upper Hessenberg:")
        Qk = QR_step!(A, k+1, k)   # internally: A ← Qₖ A
        B .= Qk * B
        Q_total .= Qk * Q_total

        py_show( "S = ", (a=A, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx;k=-2, c2=k) ), L"\quad",
                 "T = ", (b=B, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx; k=-1) ),
                     number_formatter=x->round(x,digits=2)
        )

        # ---- Right reflectors Z : clear left‑of‑diagonal entries in B
        # Start one row below the current pivot (e.g. (3,2) when k = 1)
        for i in n:-1:(k+2)
            len_i = i - k         # shrinking active length as we go up
            Z = RQ_step!(B, i, i; len=len_i)
            A .= A * Z
            Z_total .= Z_total * Z
            println("        Remove bulge in T in row  $i (len=$len_i):")
            py_show( L"\qquad\qquad S = ", (a=A, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx;k=-2, c2=k) ), L"\quad",
                     "T = ", (b=B, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx; k=-1) ),
                         number_formatter=x->round(x,digits=2)
            )

        end
    end

    return Q_total', Z_total
end
;

In [210]:
%%julia
function is_upper_hessenberg(H; tol = 1e-10)
    n = size(H, 1)
    for i in 3:n
        for j in 1:i-2
            if abs(H[i,j]) > tol
                return false
            end
        end
    end
    return true
end

function is_upper_triangular(T; tol = 1e-10)
    n = size(T, 1)
    for i in 2:n
        for j in 1:i-1
            if abs(T[i,j]) > tol
                return false
            end
        end
    end
    return true
end

function test_hessenberg_triangular_qz(n=4)
    S = randn(n, n)
    T = randn(n, n)
    A = copy(S)
    B = copy(T)

    Q, Z = hessenberg_triangular_reduction!(S, T)

    @show (A ≈ Q*S*Z')
    @show (B ≈ Q*T*Z')

    @show (Q'Q ≈ I)
    @show (Z'Z ≈ I)

    @show is_upper_hessenberg(S)
    @show is_upper_triangular(T)
    flush(stdout)
end

test_hessenberg_triangular_qz(4);


=========== Step 1 : Triangularize B from the left (QR_steps)

<IPython.core.display.Latex object>



=========== Step 2 : Hessenberg–Triangular Reduction ===
Make column 1 of A upper Hessenberg:

<IPython.core.display.Latex object>


        Remove bulge in T in row  4 (len=3):

...

Make column 2 of A upper Hessenberg:

<IPython.core.display.Latex object>


        Remove bulge in T in row  4 (len=2):

<IPython.core.display.Latex object>


A ≈ Q * S * Z' = true
B ≈ Q * T * Z' = true
Q' * Q ≈ I = true
Z' * Z ≈ I = true
is_upper_hessenberg(S) = true
is_upper_triangular(T) = true


###  stuff

In [ ]:
%%julia
# --------------------------------------------------------------------------------------------
@doc """
zero_left_of_entry!(A::Matrix{Float64}, i::Int, j::Int) -> Matrix

Applies a Householder reflection **from the right** to zero out entries
to the **left** of `A[i,j]` in row `i`, i.e., `A[i, 1:j-1]`.

This modifies `A` in place as `A ← A * Z`, where `Z` is orthogonal.

Arguments:
- `A`: Matrix to be modified in-place.
- `i`: Row index of the pivot element.
- `j`: Column index of the pivot (last element to preserve).

Returns:
- The orthogonal matrix `Z` applied from the right.
"""
function zero_left_of_entry!(A::Matrix{Float64}, i::Int, j::Int)
    n = size(A, 2)

    x     = copy(@view A[i, 1:j])
    v_sub = householder_vector(x; target = :last)

    v       = zeros(eltype(A), n)
    v[1:j] .= v_sub

    Z = I - 2 * (v * v')
    A .= A * Z

    return Z
end

# --------------------------------------------------------------------------------------------
@doc """
RZ_step!(A::Matrix{Float64}, i::Int, j::Int) -> Matrix

Applies a Householder transformation from the **right** to zero out entries
**below** the element `A[i,j]` in column `j`.

This is achieved by flipping and transposing `A` to convert the operation
into a left-side transformation.

Arguments:
- `A`: A square matrix (modified in-place).
- `i`: Row index of the pivot.
- `j`: Column index of the pivot (entries below this in column `j` are zeroed).

Returns:
- The orthogonal matrix `Z` such that `A ← A * Z` zeros below `A[i,j]`.
"""
function RZ_step!(A::Matrix{Float64}, i::Int, j::Int)
    m = size(A, 1)

    # Flip and transpose
    Jm = reverse(Matrix{Float64}(I, m, m), dims=1)
    Ã = Jm * transpose(A) * Jm

    ĩ = m - j + 1
    j̃ = m - i + 1
    Z̃ = zero_left_of_entry!(Ã, ĩ, j̃)

    # Unflip and untranspose
    A .= Jm * transpose(Ã) * Jm
    Z  = Jm * transpose(Z̃) * Jm

    return Z
end
;

In [ ]:
%%julia
# test decomposition on a random matrix of size n x n
function test_rz_decomposition(n::Int = 3)
    R  = randn(n, n)
    A  = copy(R)

    display(py_show("original matrix A = ", A, number_formatter = x -> round(x, digits=2)))

    Z = Matrix{Float64}(I, n, n)
    for j in 1:n-1
R_orig=copy(R)
        Zj  = RZ_step!(R, j, j)
@show norm(R - R_orig * Zj)
        Z   = Z * Zj # Accumulate Z = Z₁ Z₂ ⋯
        display(py_show("zeros in column $j", L"\quad\;", R, number_formatter = x -> round(x, digits=2)))
    end

    display(py_show("A = R Z = ", R, Z, number_formatter = x -> round(x, digits=2)))

    @show norm(A - R*Z) ;flush(stdout)
    @show norm(Z' * Z - I) ;flush(stdout)
end

# Run the test
test_rz_decomposition(4)

**Remarks:**
* Since $R$ is lower triangular, $\tilde{R}$ is upper triangular.
* Since $\tilde{Q}^T \tilde{Q} = \left( J Q^T J^T\right) \left( J Q J^T \right) = I$, the matrix $\tilde{Q}$ is orthogonal
* Note that for complex matrices $A$, the computation requires **the transpose of $A$** rather than the hermitian transpose.

### 3.2.3 $A = QR$ one Column at a Time

We will need the familiar $A = Q R$ decomposition as well.

Below, a naive reference implementation using Householder reflections:

In [ ]:
%%julia
# test decomposition on a random matrix of size n x n
function test_qr_decomposition(n::Int = 3)
    R  = randn(n, n)
    A  = copy(R)

    display(py_show("original matrix A = ", A, number_formatter = x -> round(x, digits=2)))

    Q_total = Matrix{Float64}(I, n, n)

    for j in 1:n-1
        Qj       = QR_step!(R, j, j)
        Q_total  = Qj * Q_total  # Accumulate Q = Q₁ Q₂ ⋯
        display(py_show("zeros in column $j", L"\quad\;", R, number_formatter = x -> round(x, digits=2)))
    end

    Q = transpose(Q_total)

    display(py_show("A = Q R = ", Q, R, number_formatter = x -> round(x, digits=2)))

    @show norm(A - Q * R) ; flush(stdout)
    @show norm(Q' * Q - I)     ; flush(stdout)
end
test_qr_decomposition(4)

In [ ]:
%%julia
"""
    QR_step!(A::Matrix{Float64}, i::Int, j::Int) -> Matrix

Applies a Householder reflection **from the left** to zero out entries
**below** the pivot `A[i,j]` in column `j`. That is, it modifies `A`
in-place as `A ← Q * A`, where `Q` is orthogonal.

Returns the orthogonal matrix `Q` such that the updated `A = Q * A`
has zeros in entries A[i+1:end, j].

Arguments:
- `A`: Matrix to modify in-place.
- `i`: Row index of the pivot.
- `j`: Column index of the pivot.

Returns:
- The orthogonal matrix `Q` such that `A ← Q * A`.
"""
function QR_step!(A::AbstractMatrix{Float64}, i::Int, j::Int)
    m = size(A, 1)

    # Extract column segment below and including the pivot
    x     = copy(@view A[i:end, j])
    v_sub = householder_vector(x; target = :first)  # Align with first coordinate

    v = zeros(eltype(A), m)
    v[i:end] .= v_sub

    Q = I - 2 * (v * v')  # Householder matrix
    A .= Q * A            # Apply from the left

    return Q
end;

## 3.3 The Generalized Schur (QZ) Decomposition

The QZ algorithm reduces a matrix pencil $(A,B)$ to its generalized (quasi-)Schur form $(S,T)$,<br>
$\qquad$ in which the generalized eigenvalues are read directly from the diagonal blocks.<br>
$\qquad$ It preserves the spectrum and records transformations so that eigenvectors can be recovered from $(S,T)$.

The **QZ algorithm** generalizes the [**Schur decomposition**](Schur_Decomposition.ipynb).

It obtains two orthogonal (or unitary) matrices $Q$ and $Z$ such that<br>
$\qquad S = Q^H A Z$ and $T = Q^H B Z = T$, where $(S,T)$

The spectrum is preserved, and the transformations $Q,Z$ allow recovery of eigenvectors from $(S,T)$:

$\qquad 
\begin{align}
A x = \lambda B x \quad & \Leftrightarrow \quad  Q A Z Z^T\ x = \lambda\ Q B Z Z^T\ x & \\
                        & \Leftrightarrow \quad  Q A Z \ \tilde{x} = \lambda\ Q B Z \ \tilde{x} \qquad\qquad & \text{ where } \tilde{x} = Z^T x
\end{align}$

Hence $(\lambda, x)$ is a generalized eigenpair of $ A x = \lambda B x$<br>
if and only if $( \lambda, Z^T x )$ is a generalized eigenpair of $ Q A Z \ \tilde{x} = \lambda \ Q B Z\ \tilde{x}$

### 3.2.1 Example Pencil

We will introduce the algorithm using the following two matrices:

In [ ]:
%%julia
A = [3.0 2.0 1.0; 2.0 3.0 4.0; 1.0 5.0 6.0]
B = [4.0 1.0 2.0; 2.0 3.0 1.0; 1.0 0.0 2.0]
py_show("Let A = ", A, L",\quad B = ", B, number_formatter=x->round_value(x,0))

### 3.2.3 The QZ Algorithm

In [ ]:
%%julia
# --------------------------------------------------------------------------------------------------------------------------
""" Column reduction helper  zero out column below A[i,j]
using a Householder reflection
"""
function apply_column_householder(AI::Matrix{T}, i::Int, j::Int) where T
    e = eltype(AI)
    function sgn(x)  x != zero(e) ? sign(x) : one(e) end

    m = size(AI, 1)
    v = AI[:, j]
    v[1:i-1] .= zero(T)                     # Zero out the upper part (above pivot)
    len_v = sqrt(v'v)

    if len_v ≈ abs(v[i])
        return nothing                      # No transformation needed
    end

    ṽ    = zeros(T, m)
    ṽ[i] = -sgn(v[i]) * len_v               # Reflected vector

    w = v + ṽ                               # Householder vector
    Q = I(m) - 2 * (w * w') / (w' * w)      # Householder matrix

    return Q * AI, Q                        # Return updated matrix and reflector
end

#    e = eltype(A)
#    AI = [copy(A) one(e) * I(m)]

# --------------------------------------------------------------------------------------------------------------------------
"""
RQ‑style “triangularize” step on a trailing block of B.
Given B_block of size m×n (n ≥ m), returns (Q_block, R_block) such that
Q_block' * B_block = R_block is upper triangular in the first column (i.e. zeroes below).
"""
function rq_triangularize_step(B_block::Matrix{Float64})
    n = size(B_block, 1)
    x = B_block[:, 1]                   # first column of the active block
    normx = norm(x)
    if normx == 0
        return 1.0Matrix(I, n, n), B_block
    end

    e1 = zeros(n); e1[1] = normx * sign(x[1])
    v = x - e1
    v /= norm(v)

    Q = I - 2 * (v * v')                # Householder reflector
    B_new = Q' * B_block                # zero below first entry in column 1
    return Q, B_new
end

# --------------------------------------------------------------------------------------------------------------------------
"""
Performs a one-column Hessenberg-style reduction (QR step) on A.

Zeros out A[k+2:end, k] by applying a Householder reflector from the right.

Returns:
- Z (full n×n orthogonal matrix),
- A_new = A * Z

Only modifies columns k+1 onward.
"""
function right_hessenberg_step(A::Matrix{Float64}, k::Int)
    n = size(A, 1)

    println("\n------------------------- Hessenberg Step k = $k -------------------------------")
    # Flip the matrix inside‑out using the true transpose
    J = reverse(Matrix{Float64}(1.0I, n, n), dims=1)
    Ã = J * transpose(A) * J
    #display(py_show("A = ", A, " Flipped A = ", Ã, number_formatter =x->round(x,digits=3)))

    R̃, Q̃ = apply_column_householder(  Ã, k+1, k)
    # display(py_show( R̃, Q̃,  number_formatter =x->round(x,digits=3)))

    if Q̃ !== nothing
        Z = J * transpose(Q̃) * J
        A = J * transpose(R̃) * J
    else
        Z = Matrix{Float64}(1.0I, n, n)
    end

    #display(py_show("Z  ", Q̃, ", ", Z, number_formatter =x->round(x,digits=2)))
    #display(py_show("final ", R̃, A, number_formatter =x->round(x,digits=2)))
    return A, Z
end
# ===================================================================================================================
# ===================================================================================================================
using Random

function test_rq_triangularize_step(n=5)
    # random B_block with more columns than rows
    m=n
    B_block = randn(m, n)
    Qb, Rb  = rq_triangularize_step(B_block)
    # Check orthogonality Qb
    errQ = norm(Qb' * Qb - I(m))
    # Check that Qb' * B_block = Rb
    errR = norm(Qb' * B_block - Rb)
    # Check first column of Rb has zeros below the first entry
    col1 = Rb[:, 1]
    errZero = norm(col1[2:end])
    println("rq test: orth err = $errQ, reconstruction err = $errR, subcolumn zeros err = $errZero")
py_show(Qb, Rb, number_formatter=x->round(x,digits=3))
end

function test_right_hessenberg_step()
    n = 5
    A = randn(n,n)
    Ai = copy(A)
    Z = Matrix{Float64}(1.0I, n, n)
    for i in 1:n-2
        Ai, Zi = right_hessenberg_step(Ai, i)
        Z = Z * Zi
    end
    display(py_show( "A -> ", Ai, ",  using Z = ", Z,  number_formatter =x->round(x,digits=3)))
    Ai ≈ A*Z
end
# --------------------------------------------------------------------------------------------------------------------------
function run_tests()
    test_rq_triangularize_step()
    test_right_hessenberg_step()
end
# --------------------------------------------------------------------------------------------------------------------------
run_tests()

In [ ]:
%%julia
###############################################################
#  QZ step 1  –  Hessenberg‑triangular reduction (safe version)
###############################################################
using LinearAlgebra

#--------------------------------------------------------------
# 1.  Safe Householder vector
#--------------------------------------------------------------
using LinearAlgebra

function householder_vector(x::AbstractVector)
    x = vec(x)
    @show x

    if all(iszero, x)
        @info "x is all zeros — returning identity vector"
        return zeros(eltype(x), length(x))
    end

    σ = norm(x)
    @show σ

    e1 = zeros(eltype(x), length(x))
    e1[1] = sign(x[1]) == 0 ? σ : sign(x[1]) * σ
    @show e1

    v = x - e1
    @show v

    vnorm = norm(v)
    @show vnorm

    if vnorm ≈ 0
        @info "v is zero — returning identity vector"
        return zeros(eltype(x), length(x))
    end

    result = v / vnorm
    @show result

    return result
end

#--------------------------------------------------------------
householder_matrix(v::AbstractVector) = I - 2*(v*v')

#--------------------------------------------------------------
# 2.  Left reflector  →  zero A[j+2:end,j]
#--------------------------------------------------------------
function apply_left_householder!(A, B, j)
    m = size(A, 1)
    if j+1 >= m
        @info "Skipping left reflector at j=$j — bottom row"
        return I
    end

    @info "Building LEFT reflector at j=$j"
    x = A[j+1:end, j]
    @show x

    v_sub = householder_vector(x)
    @show v_sub

    v = zeros(eltype(A), m)
    v[j+1:end] .= v_sub
    @show v

    Q = I - 2 * (v * v')
    A[:] = Q * A
    B[:] = Q * B

    return Q
end

#--------------------------------------------------------------
# 3.  Right reflector  →  zero B[j+1,j+2:end]
#--------------------------------------------------------------
function apply_right_householder!(A::Matrix{Float64}, B::Matrix{Float64}, j::Int)
    n = size(B, 1)

    if j+1 >= n
        @info "⏩ Skipping right reflector at column j = $j (nothing below diagonal)"
        return Matrix{Float64}(I, n, n)
    end

    println("\n🔹 APPLY RIGHT HOUSEHOLDER at column j = $j")

    # Step 1: Flip-transpose-flip the matrix to simulate a right reflector as a left one
    J = reverse(Matrix{Float64}(I, n, n), dims=1)
    B̃ = J * transpose(B) * J

    # Step 2: Identify the row in B̃ corresponding to column j in B
    # Due to flipping, column j in B corresponds to row n - j + 1 in B̃
    row = n - j + 1

    # Extract the row to reflect — we want to zero out everything except the last entry
    x = B̃[row, :]

    # Step 3: Construct a Householder reflector that maps x to ±‖x‖ · eₙ (last basis vector)
    σ = norm(x)
    if σ ≈ 0
        @info "    All-zero row — skipping reflector"
        return Matrix{Float64}(I, n, n)
    end

    e_n = zeros(n); e_n[end] = 1.0
    v = x - σ * e_n
    v /= norm(v)

    Q̃ = I - 2 * (v * v')  # Householder reflector in flipped space

    # Step 4: Convert Q̃ back to a right reflector in original space
    Z = J * transpose(Q̃) * J

    # Step 5: Apply Z to A and B from the right
    A[:] = A * Z
    B[:] = B * Z

    # Diagnostics
    println("   ⇒ ‖B[j+1:end, j]‖ = ", round(norm(B[j+1:end, j]), digits=6))
    println("   ⇒ B[:, $j] = ", round.(B[:, j], digits=4))

    return Z
end

function apply_column_householder(M::Matrix{Float64}, col::Int, start_row::Int)
    m, n = size(M)
    if start_row > m
        return M, nothing
    end

    x = M[start_row:end, col]
    if norm(x) ≈ 0
        return M, nothing
    end

    v = householder_vector(x)
    v_full = zeros(m)
    v_full[start_row:end] .= v

    Q = I - 2 * (v_full * v_full')  # Householder reflector
    R = Q * M

    return R, Q
end


#--------------------------------------------------------------
# 4.  Step 1 driver
#--------------------------------------------------------------
function qz_step1(A::AbstractMatrix{T}, B::AbstractMatrix{T}) where T<:Real
    A = Float64.(A)
    B = Float64.(B)
    n = size(A, 1)

    Qacc = Matrix{Float64}(I, n, n)
    Zacc = Matrix{Float64}(I, n, n)

    for j in 1:n-2
        @info "=== Step j = $j ==="

        Qj = apply_left_householder!(A, B, j)
        Qacc = Qj * Qacc
display(py_show(j, ": left ", A, B, number_formatter=x->round(x,digits=2)))
        Zj = apply_right_householder!(A, B, j)
display(py_show(j, ": right ", A, B, number_formatter=x->round(x,digits=2)))
        Zacc = Zacc * Zj
    end

    return A, B, Qacc, Zacc
end

###############################################################
#  Example
###############################################################
A = [
    5  2  -1   0;
    3  4   1   2;
    1 -1   3   1;
    2  0   2   4
]

B = [
    2  1  0  0;
    3  5  1  0;
    1  2  4  1;
    2  1  0  3
]

H, T, Q, Z = qz_step1(A,B)
flush(stdout)
println("A (≈ Hessenberg)")
display(round.(H, digits=4))
println("B (≈ Upper‑triangular)")
display(round.(T, digits=4))
println("‖QᵀQ − I‖ =", norm(Q'Q - I))
println("‖ZᵀZ − I‖ =", norm(Z'Z - I))

println("B = ", B)

function is_upper_triangular(M; atol=1e-12)
    m, n = size(M)
    for i in 2:m
        for j in 1:i-1
            if abs(M[i,j]) > atol
                println("  ⚠️  B[$i,$j] = ", M[i,j])
                return false
            end
        end
    end
    return true
end
@show is_upper_triangular(T)

py_show( "A_hess = ", H, ", B_tri = ", T, number_formatter =x->round(x,digits=3))


#### Step 1: Initial Reduction (Optional)

The first stage in the QZ algorithm applies orthogonal transformations to reduce the matrix pair $(A, B)$  
to a structured intermediate form:
- $A$ is reduced to **upper Hessenberg form** (zero entries below the first subdiagonal),
- $B$ is reduced to **upper triangular form**.

That is, orthogonal matrices $Q_1$ and $Z_1$ are constructed such that:  
$\qquad \displaystyle{ A_1 = Q_1^T A Z_1 }$ is upper Hessenberg  
$\qquad \displaystyle{ B_1 = Q_1^T B Z_1 }$ is upper triangular

* $Q_1$ acts from the left to annihilate entries of $A$ using a QR algorithm  
typically performed using a sequence of [**Householder reflections**](HouseholderReflections.ipynb) or [**Givens Rotations**](GivensRotations.ipynb)  
* $Z_1$ acts from the right to eliminate subdiagonal entries in $B$ using an RQ algorithm as described above.

The same transformations $Q_1$ and $Z_1$ are applied to both $A$ and $B$ to preserve the structure of the generalized eigenproblem.

No assumption is made about symmetry or definiteness of $A$ or $B$.  
The process applies to any square matrices of the same size.

In [ ]:
%%julia
matrices = naive_householder(A)
A₁       = matrices[end][2][:,1:size(A,2)]
Q₁       = matrices[end][2][:,size(A,2)+1:end]'
B₁       = Q₁ * B
h,_=nM.ge( matrices, formater=a -> string(round(a,digits=3)), Nrhs=size(A,1),
    bg_for_entries=[
        [1, 0, [[(0, 0), (2, 2)]], "yellow!30"],[1, 1, [[(0, 0), (2, 0)]], "yellow!50"],
        [2, 0, [[(1, 1), (2, 2)]], "yellow!30"],[2, 1, [[(1, 1), (2, 1)]], "yellow!50"],
    ],
    tmp_dir="/tmp")
h

In [ ]:
%%julia
py_show(L"Q_1 = ", Q₁, L"\quad A_1 = Q_1 A = ", A₁, L"\quad B_1 = Q_1 B = ", B₁, number_formatter=x->round_value(x,3))

In [ ]:
%%julia
A = [3.0 2.0 1.0 4.0;
     2.0 3.0 4.0 1.0;
     1.0 5.0 6.0 -1.0;
     1.0 2.0 1.0 3.0
]

B = [4.0 1.0 2.0 4.0;
     2.0 3.0 1.0 1.0;
     1.0 5.0 2.0 2.0;
    -1.0 3.0 1.0 4.0]
#Z,BZ = right_householder_from_row(B, 1)
Z,BZ=right_reflector_for_column(B,1)
py_show(Z, BZ, B*Z, number_formatter=x->round_value(x,2))

#### Step 2: Hessenberg–Triangular Reduction

The first stage reduces $A$ to upper Hessenberg form and $B$ to upper triangular form:  
$\qquad \displaystyle{ Q_1^T A Z_1 = H }, \qquad \displaystyle{ Q_1^T B Z_1 = R }
$

This can be done via a sequence of **Householder transformations**, similar to the QR algorithm.

#### Step 3: Iterative QZ Step

Next, an iterative process analogous to the QR algorithm is applied:
- A shift $\sigma$ is selected
- The matrix $A - \sigma B$ is implicitly factorized as $QR$
- Update:
  $\; \displaystyle{ A \leftarrow R Q + \sigma, \quad B \leftarrow Q^T B Q }$

Each iteration reduces the subdiagonal structure while maintaining the triangular form of $B$.

The process is repeated until both $A$ and $B$ are simultaneously triangular (or quasi-triangular in the complex case).

---

### Step 4: Eigenvalue Extraction

Once in triangular form  
$\qquad \displaystyle{ S = Q^T A Z }, \quad \displaystyle{ T = Q^T B Z }
$
the eigenvalues of the pencil $A - \lambda B$ are given by  
$\qquad \displaystyle{ \lambda_i = \frac{s_{ii}}{t_{ii}} }, \quad \text{for } t_{ii} \neq 0
$

Infinite or indeterminate eigenvalues may arise if $t_{ii} = 0$.

---

### Summary

- The QZ algorithm generalizes the Schur decomposition by reducing a matrix pair $(A, B)$ to triangular form via orthogonal transformations.
- The process maintains numerical stability and avoids explicit inversion.
- Eigenvalues are obtained from the resulting triangular pencil $(S, T)$.
- Left and right eigenvectors may be computed from the accumulated transformations.

This algorithm is implemented in LAPACK as `xGGEV` and accessed in SciPy via `scipy.linalg.eig`.

For full details, see:
- Golub & Van Loan, *Matrix Computations*, 4th ed., Chapter 7

Let $(A, B)$ be a pair of square matrices of the same size.  
The **generalized Schur decomposition**, also known as the **QZ decomposition**, expresses both matrices in an upper-triangular form via orthogonal (or unitary) similarity transformations.

There exist orthogonal matrices $Q$ and $Z$ such that
$\qquad \displaystyle{ Q^T A Z = S }, \quad \displaystyle{ Q^T B Z = T }$  
where $S$ and $T$ are both upper (quasi-)triangular matrices.

This defines the **generalized Schur form** of the pencil $(A, B)$.

The original generalized eigenproblem  
$\qquad \displaystyle{ A x = \lambda B x }$  
is transformed into the equivalent problem  
$\qquad \displaystyle{ S \hat{x} = \lambda T \hat{x} }$  
with $\hat{x} = Z^{-1} x$.

The generalized eigenvalues are given (formally) as:
$\qquad \displaystyle{ \lambda_i = \frac{s_{ii}}{t_{ii}} }$  
for each $i$ such that $t_{ii} \neq 0$.

If $t_{ii} = 0$ and $s_{ii} \neq 0$, then $\lambda_i = \infty$.  
If both $s_{ii}$ and $t_{ii}$ are zero, the eigenvalue is considered **undefined** or **indeterminate**, and may signal a singular pencil.

---

### Properties

- The eigenvalues of the pencil $(A, B)$ are invariant under the transformation.
- If $A$ and $B$ are both real, then $Q$, $Z$, $S$, and $T$ can be taken as real.
- If the matrices are complex, then $Q$ and $Z$ are unitary.
- The decomposition is **backward stable** and suitable for arbitrary (nonsymmetric, indefinite, or singular) pencils.

---

### Comparison with the Standard Schur Decomposition

- The **standard Schur decomposition** reduces a single matrix to triangular form:  
  $\qquad \displaystyle{ A = Q S Q^T }$
- The **generalized Schur decomposition** reduces a matrix *pair* to simultaneous triangular form via two-sided transformations:
  $\qquad \displaystyle{ (A, B) \mapsto (S, T) = (Q^T A Z, Q^T B Z) }$

This generalization is necessary because no single transformation can, in general, triangularize both $A$ and $B$ simultaneously unless they commute, which is rare in practice.

For arbitrary regular $(A,B)$, compute unitary $Q,Z$ such that  
$\qquad Q^T A Z = S, \quad Q^T B Z = T,$  
with $S,T$ (quasi)upper-triangular. This is the **generalized Schur form**.

- **Eigenvalues** are read from diagonal blocks as ratios $\qquad \lambda_i = S_{ii}/T_{ii}$ (with $T_{ii}=0$ encoding $\lambda_i=\infty$).  
- The orthogonal/unitary steps make QZ **backward stable**: computed results are exact for a nearby $(A+\Delta A,B+\Delta B)$ with small perturbations.

In [ ]:
def qz_eigs(A,B,output='real'):
    if not HAVE_SCIPY:
        raise RuntimeError("SciPy required for QZ.")
    S, T, Q, Z = qz(A, B, output=output)
    alpha = np.diag(S).astype(complex)
    beta  = np.diag(T).astype(complex)
    w = alpha / beta
    return w, (S,T,Q,Z)

# Small demo with a nonsymmetric pair
A = np.array([[4., 2., 0.],
              [1., 3., 1.],
              [0., 0., 2.]])
B = np.array([[1., 0., 0.],
              [0., 2., 0.],
              [0., 0., 1.]])

if HAVE_SCIPY:
    w, (S,T,Q,Z) = qz_eigs(A,B,output='real')
    print("QZ eigenvalues (ratios S_ii/T_ii):", np.real(w))
    # Residual check with right Schur vectors (columns of Z span invariant subspaces)
    for i in range(len(w)):
        x = Z[:,i]
        print(f"i={i}, residual:", np.round(residual(A,B,w[i],x), 3))
else:
    print("SciPy not available; skipping QZ demo.")

# 4. Detecting $\lambda=\infty$ in practice

In QZ form, $T_{ii}\approx 0$ indicates $\lambda_i=\infty$ (since $\lambda_i=S_{ii}/T_{ii}$). Equivalently, flip to the **reciprocal pencil**
$\qquad B - \mu A $
and look for eigenvalues at $\mu=0$.


In [ ]:
# Example: regular pencil with B singular -> one eigenvalue at infinity
A = np.array([[1., 0.],
              [0., 2.]])
B = np.array([[1., 0.],
              [0., 0.]])

if HAVE_SCIPY:
    w1, (S1,T1,_,Z1) = qz_eigs(A,B,output='real')
    print("Eigenvalues of (A,B):", w1, "  (one should be ∞)")
    print("T diag:", np.diag(T1))  # one ~0
    # Reciprocal
    w2, _ = qz_eigs(B,A,output='real')
    print("Eigenvalues of reciprocal (B,A):", w2, "  (expect a zero)")
else:
    print("SciPy not available; ∞-eigenvalue demo skipped.")

# 5. Reordering and deflating subspaces (via `ordqz`)

To extract a **deflating subspace** for a spectral region (e.g., $|\lambda|<1$ for discrete-time stability), reorder the generalized Schur form so that the desired eigenvalues appear first. The corresponding first columns of $Z$ span the deflating subspace.

In [ ]:
if HAVE_SCIPY:
    # Discrete-time example: select |lambda| < 1
    A = np.array([[0.8, 0.3, 0.0],
                  [0.0, 1.2, 0.2],
                  [0.0, 0.0, 0.9]])
    B = np.eye(3)

    def select_inside(alpha, beta):
        w = alpha/beta
        return np.abs(w) < 1.0

    S, T, Q, Z, alpha, beta, sdim = ordqz(A, B, sort=select_inside, output='real')
    w = alpha/beta
    print("Reordered eigenvalues:", w)
    print("Count inside unit disk:", sdim)
    # Extract deflating subspace U = Z[:, :sdim] and verify approximate invariance
    U = Z[:, :sdim]
    PA = U @ np.linalg.lstsq(U, (A@U), rcond=None)[0]
    PB = U @ np.linalg.lstsq(U, (B@U), rcond=None)[0]
    print("||A U - Proj_U(A U)||:", np.linalg.norm(A@U - PA))
    print("||B U - Proj_U(B U)||:", np.linalg.norm(B@U - PB))
else:
    print("SciPy not available; ordqz demo skipped.")


# 6. Backward error viewpoint and residual auditing

A computed pair $(\hat\lambda, \hat x)$ satisfies $A\hat x - \hat\lambda B \hat x = r$. With QZ (orthogonal/unitary updates), the algorithm is **backward stable**: it computes the exact Schur form of a nearby pencil $(A+\Delta A, B+\Delta B)$ with small perturbations.

**Practical check.** Report **normalized residuals**  
$\qquad \dfrac{\|A\hat x - \hat\lambda B\hat x\|}{\|A\|\,\|\hat x\|}$  
for all computed modes; small residuals give confidence in the solution.

In [ ]:
# Residual audit on a random example
np.random.seed(7)
A = np.random.randn(6,6)
B = np.random.randn(6,6)

if HAVE_SCIPY:
    w, (S,T,Q,Z) = qz_eigs(A,B,output='complex')
    res = np.array([residual(A,B,w[i],Z[:,i]) for i in range(len(w))])
    print("Median normalized residual:", np.median(res))
    print("Max normalized residual   :", np.max(res))
else:
    print("SciPy not available; residual audit skipped.")

# 7. Practical recipes and pitfalls

- **If $A=A^T$, $B=B^T \succ 0$**: use the **Cholesky route** (symmetric-definite). You get real eigenvalues and $B$-orthonormal eigenvectors.  
- **Otherwise (regular pencils)**: use **QZ**; read eigenvalues from $S_{ii}/T_{ii}$, detect $\lambda=\infty$ via $T_{ii}\approx 0$.  
- **Never form $B^{-1}A$ explicitly**; prefer triangular solves or Schur/QZ.  
- **Scale/equilibrate** poorly scaled inputs before solving (see Notebook 3).  
- **Always check residuals**; when in doubt, verify with multiple methods (e.g., Cholesky vs QZ) in cases where both apply.

# 8. Exercises

1) **Symmetric-definite route.** Create random SPD $B$ and symmetric $A$; solve via whitening and verify $X^T B X \approx I$ and $AX \approx BX\Lambda$.  
2) **QZ reading.** Construct a pair with a singular $B$; use QZ to detect $T_{ii}\approx 0$ and confirm the presence of $\lambda=\infty$ using the reciprocal pencil.  
3) **Reordering.** For a discrete-time model $(A,E)$, use `ordqz` to separate $|\lambda|<1$ modes and verify the deflating subspace property numerically.  
4) **Backward error.** For a random pencil, compute all residuals and relate their magnitudes to problem scaling (try equilibrating $A,B$ first).  
5) **Structure check.** For the symmetric-definite case, compare eigenvalues from the Cholesky route with those from QZ (they should agree to roundoff).

# Take Away

<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">
<!-- ----------------------------------------- Python box -->
<div style="width: 22em; height: 4.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>U, _ = np.linalg.qr(A)</div>
    <div>V, _ = np.linalg.qr(B)</div>
  </div>
</div>
<!-- ---------------------------------------- Julia box -->
<div style="width: 22em; height: 4.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>U = qr(A).Q</div>
    <div>V = qr(B).Q</div>
  </div>
</div>
</div>

In [ ]:
P      = np.fliplr(np.eye(Main.A.shape[0]))   # permutation that flips order
Qt, Rt = np.linalg.qr(P@ Main.A.T @ P)        # Qt orthogonal, Rt upper‑triangular
B = P @ Rt.T @ P                              # upper‑triangular
C = P @ Qt.T @ P                              # orthogonal

In [ ]:
Main.A - B @ C

# Why the QZ (Generalized Schur) Algorithm Works

## 1.  The Generalized Eigenvalue Problem
Given two square matrices $A,B\in\mathbb{R}^{n\times n}$ (or $\mathbb{C}^{n\times n}$),
the **pencil eigenproblem** seeks scalars $\lambda$ and non‑zero vectors
$x$ such that
$$
A\,x = \lambda\,B\,x .
\tag{1}
$$
Equivalently, $\lambda$ is a root of $\det(A-\lambda B)=0$.

## 2.  Orthogonal (Unitary) Similarities Preserve the Spectrum
For any orthogonal (real) or unitary (complex) matrices $Q,Z$ we have
$Q^{T}Q=I$ and $Z^{T}Z=I$.  Multiplying (1) on the left by $Q^{T}$
and on the right by $Z$ gives
$$
Q^{T}A Z\,\tilde{x}= \lambda\, Q^{T}B Z\,\tilde{x},
\qquad\text{where }\tilde{x}=Z^{T}x .
\tag{2}
$$
Since the map $x\mapsto\tilde{x}=Z^{T}x$ is bijective, $(\lambda,\tilde{x})$ is an
eigenpair of the transformed pencil
$$
\tilde A = Q^{T} A Z,\qquad \tilde B = Q^{T} B Z .
$$
Conversely, any eigenpair of $(\tilde A,\tilde B)$ yields an eigenpair of
$(A,B)$ by the inverse substitution $x = Z\tilde{x}$.  Hence
$$
\sigma_{\text{gen}}(A,B)=\sigma_{\text{gen}}(Q^{T} A Z,\; Q^{T} B Z).
\tag{3}
$$

## 3.  What the Transformations Do

* **Left multiplication** by $Q^{T}$ rotates the **row space**
of both matrices in exactly the same way.
* **Right multiplication** by $Z$ rotates the **column space**
of both matrices in exactly the same way.
* Because the same rotations are applied to **both** members of the
pencil, the **ratio** $A x / B x$ that defines the generalized eigenvalue
remains unchanged.


## 4.  Why Orthogonal/Unitary Matrices
Orthogonal (unitary) matrices satisfy $Q^{T}=Q^{-1}$ and $Z^{T}=Z^{-1}$;
therefore the transformation in (2) is a genuine **similarity**
transformation of the pencil.  Any invertible $S,T$ would preserve the
generalized eigenvalues, but orthogonal/unitary choices have two crucial
advantages:

1. They are norm‑preserving: $\|Q^{T}A Z\|_{2}=\|A\|_{2}$, which keeps
round‑off errors under control.
2. Their condition number is 1, guaranteeing backward stability of the
algorithm.

## 5.  The QZ Iteration
Starting from a Hessenberg–triangular pencil $(H,T)$ (obtained by an
initial reduction), a single QZ step proceeds as follows:

1. Choose a shift $\sigma$ (typically an eigenvalue of the trailing
$2\times2$ block of $(H,T)$).
2. Form the shifted pencil $(H-\sigma T,\;T)$.
3. Apply a Givens rotation $G_{1}$ from the left to zero the sub‑diagonal
element in the first column of $H-\sigma T$; simultaneously multiply both
$H$ and $T$ on the right by $G_{1}^{T}$.  This creates a **bulge**
below the Hessenberg band.
* Chase the bulge down the matrix with successive Givens rotations,
each applied on the left and on the right, until it exits at the bottom‑right
corner.

Every left/right pair $(G_{k}^{T},\,G_{k})$ is orthogonal, so by (3) the
generalized eigenvalues are unchanged.  After a full sweep the trailing
$2\times2$ block becomes (quasi‑)triangular; repeating the sweep drives the
entire pencil to the generalized Schur form
$$
Q^{T} A Z = S,\qquad Q^{T} B Z = T,
$$
with $S$ quasi‑upper‑triangular and $T$ upper‑triangular.  The eigenvalues
are then simply the ratios of corresponding diagonal (or $2\times2$ block)
entries:
$$
\lambda_{i}= \frac{s_{ii}}{t_{ii}}\quad\text{(or the ratio of the
eigenvalues of matching $2\times2$ blocks).}
$$

## 6.  Summary

* Simultaneous left and right orthogonal transformations constitute a
similarity transformation of the matrix pencil, leaving its generalized
eigenvalues invariant.
* The QZ algorithm exploits this property to reshape the pencil
(Hessenberg–triangular $\to$ quasi‑Schur) while preserving the spectrum.
* Orthogonal (unitary) matrices guarantee numerical stability because
they do not amplify norms or condition numbers.

In [ ]:
%%julia
"""
    householder_vector(x::AbstractVector; target::Symbol = :first) -> Vector

Computes a Householder vector `v` such that the reflection 
    `Q = I - 2vvᵀ` maps `x` to a vector aligned with the `target` coordinate.

Arguments:
- `x`: A vector to be reflected.
- `target`: `:first` (default) aligns with the first coordinate; `:last` aligns with the last.

Returns:
- A unit Householder vector `v` such that `Qx` has all zeros except possibly one entry.
"""
function householder_vector(x::AbstractVector; target::Symbol = :first)
    x = vec(x)
    if all(iszero, x)
        return zeros(eltype(x), length(x))
    end

    σ        = norm(x)
    e        = zeros(eltype(x), length(x))
    index    = target == :first ? 1 : length(x)
    e[index] = sign(x[index]) == 0 ? σ : sign(x[index]) * σ

    v     = x - e
    vnorm = norm(v)

    if vnorm ≈ 0
        return zeros(eltype(x), length(x))
    end

    return v / vnorm
end

"""
    QR_step!(A::Matrix{Float64}, i::Int, j::Int) -> Matrix

Applies a Householder reflection **from the left** to zero out entries
**below** the pivot `A[i,j]` in column `j`. That is, it modifies `A`
in-place as `A ← Q * A`, where `Q` is orthogonal.

Returns the orthogonal matrix `Q` such that the updated `A = Q * A`
has zeros in entries A[i+1:end, j].
"""
function QR_step!(A::Matrix{Float64}, i::Int, j::Int)
    m = size(A, 1)

    # Extract column segment below and including the pivot
    x     = copy(@view A[i:end, j])
    v_sub = householder_vector(x; target = :first)  # Align with first coordinate

    v = zeros(eltype(A), m)
    v[i:end] .= v_sub

    Q = I - 2 * (v * v')  # Householder matrix
    A .= Q * A            # Apply from the left

    return Q
end

"""
    zero_left_of_entry!(A::Matrix{Float64}, i::Int, j::Int) -> Matrix

Applies a Householder reflection **from the right** to zero out entries
to the **left** of `A[i,j]` in row `i`, i.e., `A[i, 1:j-1]`.

This modifies `A` in place as `A ← A * Z`, where `Z` is orthogonal.

Returns:
- The orthogonal matrix `Z` applied from the right.
"""
function zero_left_of_entry!(A::Matrix{Float64}, i::Int, j::Int)
    n = size(A, 2)

    x     = copy(@view A[i, 1:j])
    v_sub = householder_vector(x; target = :last)

    v       = zeros(eltype(A), n)
    v[1:j] .= v_sub

    Z = I - 2 * (v * v')
    A .= A * Z

    return Z
end
"""
    RZ_step!(A::Matrix{Float64}, i::Int, j::Int) -> Matrix

Applies a Householder transformation from the **right** to zero out entries
**below** the element `A[i,j]` in column `j`.

This is achieved by flipping and transposing `A` to convert the operation
into a left-side transformation.

Returns:
- The orthogonal matrix `Z` such that `A ← A * Z` zeros below `A[i,j]`.
"""
function RZ_step!(A::Matrix{Float64}, i::Int, j::Int)
    m = size(A, 1)

    # Flip and transpose
    Jm = reverse(Matrix{Float64}(I, m, m), dims=1)
    Ã = Jm * transpose(A) * Jm
    ĩ = m - j + 1
    j̃ = m - i + 1
    Z̃ = zero_left_of_entry!(Ã, ĩ, j̃)

    # Unflip and untranspose
    A .= Jm * transpose(Ã) * Jm
    Z  = Jm * transpose(Z̃) * Jm

    return Z
end

"""
    qz_hessenberg_triangular!(A, B)

Transforms the matrix pair (A, B) to (S, T) such that:
    S = Qᵗ A Z  is upper Hessenberg
    T = Qᵗ B Z  is upper triangular

Uses QR_step! (left Householder) and RZ_step! (right Householder)
to build the orthogonal matrices Q and Z.

Arguments:
- A, B: Square matrices, modified in-place

Returns:
- S, T: Modified A and B
- Qᵗ, Z: Orthogonal matrices (transpose of Q, and Z), such that A_orig = Q S Zᵗ, etc.
"""
function qz_hessenberg_triangular!(A::Matrix{Float64}, B::Matrix{Float64})
    n = size(A, 1)
    @assert size(B) == size(A)

    Q_total = Matrix{Float64}(I, n, n)
    Z_total = Matrix{Float64}(I, n, n)

    for j in 1:n-2
        # Step 1: Apply left Householder to zero below A[j,j]
        Qj = QR_step!(A, j, j)
        B  .= Qj * B
        Q_total .= Qj * Q_total   # ✅ left multiplication

        # Step 2: Apply right Householder to zero below B[j,j]
        Zj = RZ_step!(B, j, j)
        A  .= A * Zj
        B  .= B * Zj
        Z_total .= Zj * Z_total   # ✅ FIXED: left multiplication
    end

    S = A
    T = B
    Q = Q_total'
    Z = Z_total

    return S, T, Q, Z
end
;

In [ ]:
%%julia
function test_householder_vector()
    x = [3.0, 4.0, 0.0]
    v = householder_vector(x)
    Q = I - 2 * (v * v')
    x_trans = Q * x

    display(py_show("Original x =", x, number_formatter = x -> round(x, digits=2)))
    display(py_show("Qx =", x_trans, number_formatter = x -> round(x, digits=2)))
    @show norm(x_trans[2:end]) < 1e-10
end
test_householder_vector()

In [ ]:
%%julia
function test_QR_step()
    A = randn(4, 4)
    A_orig = copy(A)
    Q = QR_step!(A, 1, 1)

    display(py_show("A after QR_step! =", A, number_formatter = x -> round(x, digits=2)))
    @show norm(A[2:end, 1]) < 1e-10
    @show Q' * Q ≈ I
end
test_QR_step();

In [ ]:
%%julia
"""
    test_qr_decomposition(n)

Test QR decomposition using left Householder reflections (QR_step!)
on a random matrix A to produce:
    A = Q * R
"""
function test_qr_decomposition(n::Int = 4)
    A = randn(n, n)
    R = copy(A)

    display(py_show("original matrix A = ", A, number_formatter = x -> round(x, digits=2)))

    Q = Matrix{Float64}(I, n, n)
    for j in 1:n
        R_orig = copy(R)
        Qj     = QR_step!(R, j, j)
        @show norm(R - Qj * R_orig)
        Q      = Qj * Q  # Accumulate from left
        display(py_show("zeros in column $j", R, number_formatter = x -> round(x, digits=2)))
    end

    display(py_show("A = Q R = ", Q', R, number_formatter = x -> round(x, digits=2)))

    @show norm(A - Q' * R)           # A = Qᵗ R → Q R ≈ A
    @show norm(Q' * Q - I)           # Q should be orthogonal
end
test_qr_decomposition(4)

In [ ]:
%%julia
function test_zero_left_of_entry()
    R = randn(4, 4)
    A = copy(R)
    Z = zero_left_of_entry!(R, 2, 3)

    display(py_show("R =", R, number_formatter = x -> round(x, digits=2)))
    @show norm(R[2, 1:2]) < 1e-10;  flush(stdout)
    @show norm(A-R*Z);              flush(stdout)
    @show Z' * Z ≈ I
end
test_zero_left_of_entry();

In [ ]:
%%julia
function test_RZ_step()
    R = randn(4, 4)
    A = copy(R)

    Z = RZ_step!(R, 1, 2)  # Apply RZ step to zero below A[1,2]

    display(py_show("R = ", R, number_formatter = x -> round(x, digits=2)))
    display(py_show("Z = ", Z, number_formatter = x -> round(x, digits=2)))

    @show norm(R[2:end, 2])
    @show norm(Z' * Z - I)
    @show norm(A - R * Z)
    @show norm(A - R * Z')
    @show norm(A - Z * R)
    @show norm(A - Z' * R)
end
test_RZ_step()

In [ ]:
%%julia
"""
    test_rz_decomposition(n)

Test RZ decomposition using right Householder reflections (RZ_step!)
on a random matrix A to produce:
    A = R * Z
"""
function test_rz_decomposition(n::Int = 4)
    A = randn(n, n)
    R = copy(A)

    display(py_show("original matrix A = ", A, number_formatter = x -> round(x, digits=2)))

    Z = Matrix{Float64}(I, n, n)
    for j in 1:n-1
        R_orig = copy(R)
        Zj     = RZ_step!(R, j, j)
        @show norm(R - R_orig * Zj)
        Z      = Z * Zj  # Accumulate from right
        display(py_show("zeros in column $j", R, number_formatter = x -> round(x, digits=2)))
    end

    display(py_show("A = R Z = ", R, Z, number_formatter = x -> round(x, digits=2)))

    @show norm(A - R * Z)
    @show norm(Z' * Z - I)
end
test_rz_decomposition(4)

In [ ]:
%%julia
function test_qz_hessenberg_triangular(n::Int = 4)
    A = randn(n, n)
    B = randn(n, n)
    A_orig = copy(A)
    B_orig = copy(B)

    S, T, Q, Z = qz_hessenberg_triangular!(A, B)
@show S
@show T

    display(py_show("S = Qᵗ A Z =", S, number_formatter = x -> round(x, digits=2)))
    display(py_show("T = Qᵗ B Z =", T, number_formatter = x -> round(x, digits=2)))

    @show norm(Q' * A_orig * Z - S)
    @show norm(Q' * B_orig * Z - T)
    @show norm(Q' * Q - I)
    @show norm(Z' * Z - I)

    # Check structure
    is_hessenberg = all(S[i, j] ≈ 0 for i in 3:n for j in 1:i-2)
    is_upper_tri  = all(T[i, j] ≈ 0 for i in 2:n for j in 1:i-1)

    @show is_hessenberg
    @show is_upper_tri
end
test_qz_hessenberg_triangular()

In [ ]:
%%julia

"""
    schur_generalized(A, B; show_steps=false)

Compute the generalized Schur decomposition (QZ decomposition) of (A, B).

Returns (S, T, Q, Z) such that:
    Q'*A*Z = S  (quasi upper-triangular)
    Q'*B*Z = T  (upper-triangular)

Supports real or complex matrices.
"""
function schur_generalized(A::AbstractMatrix{Elt}, B::AbstractMatrix{Elt};
                           show_steps::Bool = false) where Elt <: Union{Real, Complex}

    n = size(A,1)
    n == size(A,2) || throw(ArgumentError("A must be square"))
    size(B) == size(A) || throw(ArgumentError("A and B must be the same size"))

    A = copy(A)
    B = copy(B)

    if show_steps
        println("Step 1: QR decomposition of B")
    end
    F = qr(B)
    R = F.R
    Qb = Matrix(F.Q)

    if show_steps
        println("Step 2: Apply Qᵀ to A")
    end
    A_hat = Qb' * A

    if show_steps
        println("Step 3: Initialize Q and Z")
    end
    Q = Qb
    Z = Matrix{Elt}(I, n, n)

    if show_steps
        println("Step 4: Reduce (A, B) to generalized Hessenberg form")
    end
    A_hess, B_hess, Q, Z = reduce_to_generalized_hessenberg!(A_hat, R, Q, Z; show_steps)

    if show_steps
        println("Step 5: Use QZ iteration")
    end
    S, Tmat = simple_qz_iteration(A_hess, B_hess)  # Placeholder for now

    return S, Tmat, Q, Z
end

"""
    reduce_to_generalized_hessenberg!(A, B, Q, Z; show_steps=false)

Reduces (A, B) to generalized Hessenberg form:
A becomes upper Hessenberg, B becomes upper-triangular.
"""
function reduce_to_generalized_hessenberg!(A, B, Q, Z; show_steps=false)
    n = size(A, 1)

    for j in 1:n-2
        for i in n:-1:j+2
            if show_steps
                println("  Givens rotation on rows ($i-1,$i) for A[:, $j]")
            end
            G = givens_rotation(A[i-1, j], A[i, j])
            A[[i-1, i], :] .= G * A[[i-1, i], :]
            B[[i-1, i], :] .= G * B[[i-1, i], :]
            Q[:, [i-1, i]] .= Q[:, [i-1, i]] * G'

            if show_steps
                println("  Givens rotation on columns ($i,$i-1) for B")
            end
            G2 = givens_rotation(B[i, i-1], B[i-1, i-1])
            A[:, [i-1, i]] .= A[:, [i-1, i]] * G2
            B[:, [i-1, i]] .= B[:, [i-1, i]] * G2
            Z[:, [i-1, i]] .= Z[:, [i-1, i]] * G2
        end
    end
    return A, B, Q, Z
end

using LinearAlgebra, Random

function givens_real(a::Float64, b::Float64)
    if b == 0.0
        return (1.0, 0.0)
    else
        r = hypot(a, b)
        c = a / r
        s = b / r
        return (c, s)
    end
end

function apply_givens_left_real!(A::Matrix{Float64}, i::Int, k::Int, c::Float64, s::Float64)
    ncol = size(A, 2)
    for j in 1:ncol
        temp = c * A[i, j] + s * A[k, j]
        A[k, j] = -s * A[i, j] + c * A[k, j]
        A[i, j] = temp
    end
end

function apply_givens_right_real!(A::Matrix{Float64}, i::Int, k::Int, c::Float64, s::Float64)
    nrow = size(A, 1)
    for j in 1:nrow
        temp = c * A[j, i] + s * A[j, k]
        A[j, k] = -s * A[j, i] + c * A[j, k]
        A[j, i] = temp
    end
end

function qz_real(A::Matrix{Float64}, B::Matrix{Float64};
                 max_iter::Int = 50, tol::Float64 = 1e-10, debug::Bool=true)
    n = size(A,1)
    S = copy(A)
    T = copy(B)
    Q = Matrix{Float64}(I, n, n)
    Z = Matrix{Float64}(I, n, n)

    for iter in 1:max_iter
        if debug
            println("▶ QZ Iteration $iter")
        end

        # Compute simple shift μ: use S[n,n]/T[n,n]
        μ = S[n,n] / T[n,n]

        # Introduce bulge: Givens on first column to zero S[2,1] in S - μ T
        f = S[1,1] - μ*T[1,1]
        g = S[2,1] - μ*T[2,1]
        (c0, s0) = givens_real(f, g)
        # apply left rotation at rows (1,2)
        apply_givens_left_real!(S, 1, 2, c0, s0)
        apply_givens_left_real!(T, 1, 2, c0, s0)
        apply_givens_left_real!(Q, 1, 2, c0, s0)

        for k in 1:(n-1)
            # Right rotation to zero T[k+1, k]
            (c1, s1) = givens_real(T[k,k], T[k+1,k])
            apply_givens_right_real!(S, k, k+1, c1, s1)
            apply_givens_right_real!(T, k, k+1, c1, s1)
            apply_givens_right_real!(Z, k, k+1, c1, s1)

            # If not at last, Left rotation to zero S[k+1, k]
            if k < n-1
                (c2, s2) = givens_real(S[k+1, k], S[k+2, k])
                apply_givens_left_real!(S, k+1, k+2, c2, s2)
                apply_givens_left_real!(T, k+1, k+2, c2, s2)
                apply_givens_left_real!(Q, k+1, k+2, c2, s2)
            end
        end

        # Clean subdiagonals
        # T: zero T[i+1,i] if small
        maxTsub = 0.0
        for i in 1:(n-1)
            if abs(T[i+1,i]) < tol*(abs(T[i,i]) + abs(T[i+1,i+1]))
                T[i+1,i] = 0.0
            end
            maxTsub = max(maxTsub, abs(T[i+1,i]))
        end

        # S: zero S[i+2,i] if small
        maxS2sub = 0.0
        for i in 1:(n-2)
            if abs(S[i+2,i]) < tol*(abs(S[i+1,i+1]) + abs(S[i+2,i+2]))
                S[i+2,i] = 0.0
            end
            maxS2sub = max(maxS2sub, abs(S[i+2,i]))
        end

        resA = norm(Q' * A * Z - S)
        resB = norm(Q' * B * Z - T)

        if debug
            println("‣ Residual norm A: $resA")
            println("‣ Residual norm B: $resB")
            println("‣ max |T[i+1,i]| = $maxTsub")
            println("‣ max |S[i+2,i]| = $maxS2sub")
            println("‣ Preview S =")
            println(round.(S, digits=2))
            println("‣ Preview T =")
            println(round.(T, digits=2))
            println("────────────────────────────────────────────")
        end

        if maxTsub < tol && maxS2sub < tol
            if debug
                println("✅ QZ converged at iteration $iter")
            end
            break
        end
    end

    return S, T, Q, Z
end

In [ ]:
%%julia
using LinearAlgebra

# ----------------------------------------
# 1. Givens rotation constructor
# ----------------------------------------
"""
Returns the Givens rotation parameters (c, s)
such that [c s; -s c] * [a; b] = [r; 0]
"""
function givens_real(a::Float64, b::Float64)
    if b == 0.0
        return (1.0, 0.0)
    else
        r = hypot(a, b)
        c = a / r
        s = b / r
        return (c, s)
    end
end

# ----------------------------------------
# 2. Apply Givens from the left (affects rows i and k)
# ----------------------------------------
function apply_givens_left!(A::Matrix{Float64}, i::Int, k::Int, c::Float64, s::Float64)
    n = size(A, 2)
    for j in 1:n
        temp = c * A[i, j] + s * A[k, j]
        A[k, j] = -s * A[i, j] + c * A[k, j]
        A[i, j] = temp
    end
end

# ----------------------------------------
# 3. Apply Givens from the right (affects columns i and k)
# ----------------------------------------
function apply_givens_right!(A::Matrix{Float64}, i::Int, k::Int, c::Float64, s::Float64)
    n = size(A, 1)
    for j in 1:n
        temp = c * A[j, i] + s * A[j, k]
        A[j, k] = -s * A[j, i] + c * A[j, k]
        A[j, i] = temp
    end
end

# ----------------------------------------
# 4. QZ Algorithm: Reduce (A, B) to Generalized Schur Form
# ----------------------------------------
function qz_schur_real(A::Matrix{Float64}, B::Matrix{Float64};
                       max_iter::Int = 100, tol::Float64 = 1e-10, debug::Bool = true)

    n = size(A, 1)

    # Step 1: Reduce A to Hessenberg, B to triangular
    F = hessenberg(A)  # A = Q₀ H Q₀ᵀ
    G = qr(B)          # B = Q₁ R

    S = copy(F.H)      # S = Q₀ᵀ A Q₀
    T = copy(G.R)      # T = Q₁ᵀ B
    Q = Matrix(F.Q)    # Accumulate Q
    Z = Matrix(G.Q)    # Accumulate Z

    # Step 2: Iterative QZ
    for iter in 1:max_iter
        if debug
            println("▶ Iteration $iter")
        end

        # Simple shift strategy: μ = S[n,n] / T[n,n]
        μ = S[n,n] / T[n,n]

        # Step 1: Bulge introduction — zero S[2,1] in S - μT
        f = S[1,1] - μ * T[1,1]
        g = S[2,1] - μ * T[2,1]
        (c, s) = givens_real(f, g)

        apply_givens_left!(S, 1, 2, c, s)
        apply_givens_left!(T, 1, 2, c, s)
        apply_givens_left!(Q, 1, 2, c, s)

        # Step 2: Bulge chase
        for k in 1:(n-1)
            # Right rotation to zero T[k+1, k]
            (cR, sR) = givens_real(T[k,k], T[k+1,k])
            apply_givens_right!(S, k, k+1, cR, sR)
            apply_givens_right!(T, k, k+1, cR, sR)
            apply_givens_right!(Z, k, k+1, cR, sR)

            # Left rotation to zero S[k+1, k] (if not last)
            if k < n - 1
                (cL, sL) = givens_real(S[k+1,k], S[k+2,k])
                apply_givens_left!(S, k+1, k+2, cL, sL)
                apply_givens_left!(T, k+1, k+2, cL, sL)
                apply_givens_left!(Q, k+1, k+2, cL, sL)
            end
        end

        # Step 3: Clean subdiagonal entries (deflation check)
        for i in 1:(n-1)
            if abs(T[i+1,i]) < tol*(abs(T[i,i]) + abs(T[i+1,i+1]))
                T[i+1,i] = 0.0
            end
            if abs(S[i+1,i]) < tol*(abs(S[i,i]) + abs(S[i+1,i+1]))
                S[i+1,i] = 0.0
            end
        end

        # Step 4: Convergence check
        resA = norm(Q' * A * Z - S)
        resB = norm(Q' * B * Z - T)
        if debug
            println("‣ Residual ‖QᵗAQ - S‖ = $resA")
            println("‣ Residual ‖QᵗBQ - T‖ = $resB")
            println("-------------------------------------")
        end

        if resA < tol && resB < tol
            println("✅ Converged in $iter iterations.")
            break
        end
    end

    return S, T, Q, Z
end

A = [1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 10.0]
B = [2.0 1.0 0.0; 0.0 3.0 1.0; 1.0 0.0 4.0]

S, T, Q, Z = qz_schur_real(A, B)

# Verify:
Q' * A * Z ≈ S
Q' * B * Z ≈ T

flush(stdout)
py_show( S, T, number_formatter=x->round(x,digits=2))

In [115]:
%%julia

function RQ_step!(
    A::AbstractMatrix{Float64},
    i::Int,              # pivot row index
    j::Int;              # pivot column index
    len::Int = j         # number of active entries (default: full 1:j)
)
    n = size(A, 2)
    if len <= 0
        return Matrix{Float64}(I, n, n)
    end

    start_col = max(1, j - len + 1)
    x = @view A[i, start_col:j]          # read directly from A

    # nothing to eliminate?
    #if all(iszero, x)
    #    return Matrix{Float64}(I, n, n)
    #end

    v_sub = householder_vector(x; target = :last)

    v = zeros(eltype(A), n)
    v[start_col:j] .= v_sub

    Qr = I - 2 * (v * v')
    A .= A * Qr
    return Qr
end


function test_RQ_step!(i=3,j=3, l=j)
    println("▶ Testing RQ_step! ...")
    f(x)=round(x,digits=2)

    # Random dense matrix
    A = randn(4,4)
    A0 = copy(A)

    # Apply the RQ step (zeros entries left of pivot)
    Qr = RQ_step!(A, i, j; len=l)
py_show(A, number_formatter=f)

    println("Original A = ")
    display(round.(A0, digits=3))
    println("\nAfter RQ_step!(A, $i, $j):")
    display(round.(A, digits=3))
    println("\nReflection Qr = ")
    display(round.(Qr, digits=3))

    # --- check orthogonality -------------------------------------------------
    err_orth = norm(Qr' * Qr - I(size(Qr,1)))
    println("‣ Orthogonality error = ", err_orth)

    # --- check zeroing pattern ----------------------------------------------
    left_segment = A[i, 1:j-1]
    println("‣ Left-segment norm (should be ≈ 0) = ", norm(left_segment))

    # --- confirm norm preservation ------------------------------------------
    diff_norm = abs(norm(A) - norm(A0))
    println("‣ Frobenius norm diff = ", diff_norm)

    # --- qualitative results summary ----------------------------------------
    println("\n✅ Summary:")
    println("Orthogonal? ", err_orth < 1e-12)
    println("Left-of-pivot entries zero? ", norm(left_segment) < 1e-12)
    println("Frobenius norm preserved? ", diff_norm < 1e-12)
end

# Run the test
test_RQ_step!(1,4,2)


▶ Testing RQ_step! ...

<IPython.core.display.Latex object>


Original A = 

After RQ_step!(A, 1, 4):

Reflection Qr = 
‣ Orthogonality error = 6.700006661346569e-16
‣ Left-segment norm (should be ≈ 0) = 0.6349642266225191
‣ Frobenius norm diff = 4.440892098500626e-16

✅ Summary:
Orthogonal? true
falseof-pivot entries zero? 
trueenius norm preserved? 

In [120]:
%%julia
"""
Educational Hessenberg–Triangular reduction for a matrix pair (A,B).

Uses:
  • QR_step!(A, i, j) – left Householder; zeros entries below A[i,j].
  • RQ_step!(A, i, j) – right Householder; zeros entries left of A[i,j].

Both functions modify the given matrix in place and return the full N×N orthogonal Q or Z.

Performs:
    Qᵗ * A * Z = H
    Qᵗ * B * Z = T
where H is upper Hessenberg and T is upper triangular.
"""
function hessenberg_triangular_reduction!(A::Matrix{Float64}, B::Matrix{Float64})
    n = size(A, 1)
    Q_total = Matrix{Float64}(I, n, n)
    Z_total = Matrix{Float64}(I, n, n)

    # ---------------------------------------------------------
    # Step 1 : Triangularize B (QR-type left transformations)
    # ---------------------------------------------------------
    println("=== Step 1 : Triangularize B ===")
    for j in 1:n-1
        Q = QR_step!(B, j, j)      # internally: B ← Q B
        A .= Q * A                 # apply Q to A once
        Q_total .= Q * Q_total
        println("After column $j elimination in B:")
        display(py_show(round.(A,digits=2), round.(B,digits=2)))
    end

    # ---------------------------------------------------------
    # Step 2 : Hessenberg–Triangular Reduction
    # ---------------------------------------------------------
    println("\n=== Step 2 : Hessenberg–Triangular Reduction ===")
    for k in 1:n-2
        # ---- Left reflector Qₖ: zero below A[k+1,k]
        Qk = QR_step!(A, k+1, k)   # internally: A ← Qₖ A
        B .= Qk * B
        Q_total .= Qk * Q_total
        println("After left Householder at k = $k:")
        display(py_show(round.(A,digits=2), round.(B,digits=2)))

        # ---- Right reflectors Z : clear left‑of‑diagonal entries in B
        # Start one row below the current pivot (e.g. (3,2) when k = 1)
        for i in n:-1:(k+2)
            len_i = i - k         # shrinking active length as we go up
            Z = RQ_step!(B, i, i; len=len_i)
            A .= A * Z
            Z_total .= Z_total * Z
            println("After right Householder at row $i (len=$len_i):")
            display(py_show(round.(A,digits=2), round.(B,digits=2)))
        end
    end

    # ---------------------------------------------------------
    # Step 3 : Verification output
    # ---------------------------------------------------------
    println("\n✅ Final Results:")
    H = Q_total' * A * Z_total
    T = Q_total' * B * Z_total
    println("H = Qᵗ A Z (≈ upper Hessenberg)")
    println("T = Qᵗ B Z (≈ upper triangular)")
    display(py_show(round.(H,digits=4), round.(T,digits=4)))
    return H, T, Q_total', Z_total
end



########################################################################################################
function is_upper_hessenberg(H; tol = 1e-10)
    n = size(H, 1)
    for i in 3:n
        for j in 1:i-2
            if abs(H[i,j]) > tol
                return false
            end
        end
    end
    return true
end

function is_upper_triangular(T; tol = 1e-10)
    n = size(T, 1)
    for i in 2:n
        for j in 1:i-1
            if abs(T[i,j]) > tol
                return false
            end
        end
    end
    return true
end

function test_hessenberg_triangular_qz(n=4)
    S = randn(n, n)
    T = randn(n, n)
    A = copy(S)
    B = copy(T)

    H, T, Q, Z = hessenberg_triangular_reduction!(A, B)

    println("\n✅ Final Matrices:")
    println("H = Qᵗ A Z = "); println(round.(H, digits=4))
    println("T = Qᵗ B Z = "); println(round.(T, digits=4))
    flush(stdout)

    # Residuals
    resA = norm(Q' * A * Z - H)
    resB = norm(Q' * B * Z - T)

    println("\n✅ Properties:")
    println("‣ Q orthogonal? ", norm(Q' * Q - I) < 1e-10)
    println("‣ Z orthogonal? ", norm(Z' * Z - I) < 1e-10)
    println("‣ H is upper Hessenberg? ", is_upper_hessenberg(H))
    println("‣ T is upper triangular? ", is_upper_triangular(T))
    println("‣ Residual ‖Qᵗ A Z - H‖ = ", resA)
    println("‣ Residual ‖Qᵗ B Z - T‖ = ", resB)
    flush(stdout)
H,T
end


# Run the test
H,T=test_hessenberg_triangular_qz()

py_show( H, T, number_formatter=x->round(x,digits=2))

=== Step 1 : Triangularize B ===
After column 1 elimination in B:


<IPython.core.display.Latex object>

After column 2 elimination in B:

<IPython.core.display.Latex object>


After column 3 elimination in B:


...

<IPython.core.display.Latex object>



✅ Final Results:
H = Qᵗ A Z (≈ upper Hessenberg)
T = Qᵗ B Z (≈ upper triangular)

<IPython.core.display.Latex object>



✅ Final Matrices:
H = Qᵗ A Z = 
[1.0507 0.9951 -0.6191 0.7456; -1.8051 -0.3869 -1.0678 -1.3228; -0.3884 -1.256 -0.7173 1.3302; -1.7575 -0.7534 -0.6757 -1.0657]
T = Qᵗ B Z = 
[-1.0321 -0.54 1.3454 0.6198; -0.6448 -1.2102 0.4566 0.7133; -0.5733 -0.4499 1.1232 -0.7005; -0.8648 0.0019 0.9348 -0.962]

✅ Properties:
trueorthogonal? 
trueorthogonal? 
falses upper Hessenberg? 
falses upper triangular? 
6.625637212939381Z - H‖ = 
4.641895639407208Z - T‖ = 


<IPython.core.display.Latex object>